In [ ]:
import json
import pandas as pd
from coinmarketcap import Market
# function that checks for NaN in a pandas df
def check_nan(df):
    count = 0
    for i,j in enumerate(df.isnull().sum()):
        if j != 0:
            print(df.columns[i], "contains {} NaN".format(j))
            count += 1
    if count == 0:
        print("no NaN")


coinmarketcap = Market()
# the output of below call is byte so we need to convert it to str
a = coinmarketcap.ticker()
# conversion bytecode to str
json_thing = a.decode()
json_thing = json.loads(json_thing)
df = pd.DataFrame(json_thing)
df.last_updated = pd.to_datetime(df.last_updated, unit='s')
del df['available_supply']
del df['id']
del df['percent_change_1h']
del df['percent_change_7d']
del df['name']
del df['total_supply']

df = df[['24h_volume_usd', 'symbol', 'last_updated', 'market_cap_usd',
       'percent_change_24h', 'price_btc', 'price_usd', 'rank']]
check_nan(df)
df.dropna(inplace=True)
check_nan(df)
# df.sort_values('24h_volume_usd', ascending=False, inplace=True)
json_thing

In [140]:
# limit = 2000 <> rows 2000
# from 2014-10-11 03:00:00
# to 2015-01-02 10:00:00

print(r.json()['TimeFrom'])
print(r.json()['TimeTo'])

1413021600
1420192800


In [179]:
import datetime
print("Date From",datetime.datetime.fromtimestamp(r.json()['TimeFrom']))
print("Date To", datetime.datetime.fromtimestamp(r.json()['TimeTo']))

date_from  = datetime.datetime(2015, 1, 2, 10, 00, 00).timestamp()
date_from

Date From 2014-10-11 04:00:00
Date To 2015-01-02 10:00:00


1420192800.0

In [170]:
datetime.datetime.fromtimestamp(1420196399)

datetime.datetime(2015, 1, 2, 10, 59, 59)

In [268]:
import datetime
import time
dt = parser.parse("2014-10-11 04:00:00")
time.mktime(dt.timetuple())

1412982000.0

In [160]:
from dateutil import parser
dt = parser.parse("2014-10-11 04:00:00")

### Historical data extraction with help of cryptocompare.com API
https://www.cryptocompare.com/api/#introduction

In [28]:
##### Hourly data from cryprocompare

import pandas as pd
import datetime
import requests
from IPython.display import display

# generic url for data extraction from cryptocompare.com's API
url = 'https://min-api.cryptocompare.com/data/{0}?fsym={1}&tsym={2}&limit={3}&toTs={4}&e={5}'

# parameters for the url
# data resolution {0}
hourly_data = "histohour"
minute_data = "histominute"
# currency from {1}
curr_from = "BTC"
# currency to {2}
curr_to = "EUR"

############################################################
# limit of hours with maximum 2000 (1992 is 83 days) {3}
# calculating and converting time for data extraction {4}
date_from_datetime  = datetime.datetime(2017, 6, 1, 0, 0)
print(date_from_datetime)

# enter date_to that is <= than 83 days from date_from for hourly data or <= 83 hr if minute data
date_to_datetime = datetime.datetime(2017, 6, 6, 0, 0)
print(date_to_datetime)

# checking that the entered dates have no more than 2000 hr between them
delta_in_hours = int(((date_to_datetime - date_from_datetime).total_seconds() / 3600))
print("There are {0} hrs or {1:.2f} days between the dates {2} and {3}".format(delta_in_hours, delta_in_hours / 24, \
      date_from_datetime, date_to_datetime))
limit_hours = 2000
assert (delta_in_hours < limit_hours), \
"There must be <= {0} hrs between dates, but the current value is {1}. Change the dates interval"\
.format(limit_hours, delta_in_hours)
    
# calculating the the date_to by adding 1999 hr to the date_from
# date_to = (date_from_datetime + datetime.timedelta(hours=2000-1))
date_to_timestamp = date_to_datetime.timestamp()
print("The Date To timestamp is {}".format(date_to_timestamp))
############################################################

# coin exchange name, if not used, then CCAGG  {5}
exchange = "Kraken"#"&e=Kraken" (default exchange is CCCAGG)



r = requests.get(url.format(minute_data, curr_from, curr_to, delta_in_hours, date_to_timestamp, exchange))

df = pd.DataFrame(r.json()['Data'])
df.time = pd.to_datetime(df.time, unit='s')
if exchange == "": exchange = "CCCAGG"
print("Prices from the exchange: {}".format(exchange))

print("##########################")
df.set_index(df['time'], drop=True, inplace=True)
del df['time']
display(df.head(5))
display(df.tail(5))

2017-06-01 00:00:00
2017-06-06 00:00:00
There are 120 hrs or 5.00 days between the dates 2017-06-01 00:00:00 and 2017-06-06 00:00:00
The Date To timestamp is 1496703600.0
Prices from the exchange: Kraken
##########################


,close,high,low,open,volumefrom,volumeto
time,,,,,,
2017-06-05 21:00:00,2358.67,2359.02,2355.54,2355.96,2.3400,5502.62
2017-06-05 21:01:00,2359.02,2359.02,2355.89,2358.67,1.8700,4419.74
2017-06-05 21:02:00,2359.02,2359.02,2355.96,2359.02,0.3479,820.38
2017-06-05 21:03:00,2359.02,2359.02,2359.01,2359.02,2.1300,5014.99
2017-06-05 21:04:00,2359.02,2359.02,2359.01,2359.02,0.5619,1325.47


,close,high,low,open,volumefrom,volumeto
time,,,,,,
2017-06-05 22:56:00,2376.66,2379.00,2376.66,2376.75,7.35,17470.81
2017-06-05 22:57:00,2374.88,2376.67,2368.05,2376.66,16.48,39097.93
2017-06-05 22:58:00,2373.16,2374.88,2369.00,2374.88,5.78,13719.27
2017-06-05 22:59:00,2368.39,2373.16,2368.39,2373.16,10.43,24731.78
2017-06-05 23:00:00,2371.03,2371.03,2366.26,2368.39,5.29,12533.87


In [20]:
from flask import Flask
app = Flask (__name__)

@app.route("/")
def hello():
    return "hello me!"

if __name__ == "__main__":
    app.run()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jun/2017 20:25:06] "GET / HTTP/1.1" 200 -


In [24]:
l1

[2527.42, 2527.42, 2528.92, 2528.92, 2529.12, 2529.12, 2529.12, 2528.8]

In [9]:
import rethinkdb as r
import pandas as pd
import json
r.connect( "localhost", 28015).repl()

In [8]:
authors = r.db("test").table("authors").run()
authors

<rethinkdb.net.DefaultCursor object at 0x7f39e7ded7b8 (done streaming):
 [{'posts': [{'title': 'Civil rights', 'content': "There are some words I've known since..."}], 'name': 'Jean-Luc Picard', 'id': '56d53d54-7ef4-41b7-8846-7619eefae777', 'tv_show': 'Star Trek TNG'}, {'posts': [{'title': 'Decommissioning speech', 'content': 'The Cylon War is long over...'}, {'title': 'We are at war', 'content': 'Moments ago, this ship received...'}, {'title': 'The new Earth', 'content': 'The discoveries of the past few days...'}], 'name': 'William Adama', 'id': '68ce53bd-9e13-4fc0-96f9-923cc7b752f9', 'tv_show': 'Battlestar Galactica'}, {'posts': [{'title': 'The oath of office', 'content': 'I, Laura Roslin, ...'}, {'title': 'They look like us', 'content': 'The Cylons have the ability...'}], 'name': 'Laura Roslin', 'id': 'dfdc8db5-d1fd-4ea9-ad81-a6b88149b463', 'tv_show': 'Battlestar Galactica'}]>

In [12]:
for i in authors:
    print(i.json())

In [18]:
#r.db_create("COIN_PRICES").run()
r.db("COIN_PRICES").table_create("BTCUSD").run()
r.db("COIN_PRICES").table("BTCUSD").insert({'USD': 2510.01}).run()

{'deleted': 0,
 'errors': 0,
 'generated_keys': ['7ad51e50-6bbd-4110-b53b-770869267f97'],
 'inserted': 1,
 'replaced': 0,
 'skipped': 0,
 'unchanged': 0}

In [289]:
import datetime
datetime.datetime.fromtimestamp(1420196399)

import datetime
import time
dt = parser.parse("2014-10-11 04:00:00")
time.mktime(dt.timetuple())

tmp = pd.to_datetime(1496839680, unit="s")
tmp.to_pydatetime()


Timestamp('2017-06-07 12:48:00')

In [354]:
import time
import datetime
import requests
import rethinkdb as r
import pandas as pd

r.connect( "localhost", 28015).repl()

url = 'https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=USD&limit=0&aggregate=0&e=Kraken'
def data_extract(url):
    l1 = list()
    while True:
        url_output = requests.get(url).json()['Data']
        url_output[0]['time'] = datetime.datetime.isoformat(datetime.datetime.fromtimestamp(url_output[0]['time']))
        url_output[1]['time'] = datetime.datetime.isoformat(datetime.datetime.fromtimestamp(url_output[1]['time']))
        #df = pd.DataFrame(url_output)
        #df.time = pd.to_datetime(df.time, unit='s')

        

        #display(df)
        r.db("COIN_PRICES").table("BTCUSD").insert(url_output[0]).run()
        time.sleep(60)
        

data_extract(url)  

ConnectionError: HTTPSConnectionPool(host='min-api.cryptocompare.com', port=443): Max retries exceeded with url: /data/histominute?fsym=BTC&tsym=USD&limit=0&aggregate=0&e=Kraken (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f39fc39d390>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [349]:
from IPython.display import display
import numpy as np
np.set_printoptions(suppress=True)
from_db = r.db("COIN_PRICES").table("BTCUSD").run()
df_from_db = pd.DataFrame()
for i in from_db:

    df_from_db = df_from_db.append(pd.io.json.json_normalize(i))

#df_from_db.time = pd.to_datetime(df_from_db.time, unit='s')
# df_from_db.set_index(df_from_db.time, inplace=True)
# df_from_db.drop(['time'],inplace=True)
df_from_db.sort(columns='time')

/home/yol/miniconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  del sys.path[0]


,close,high,id,low,open,time,volumefrom,volumeto
0,2700.00,2701.00,6298d4be-86b8-4f11-bfaa-3295832943a6,2700.00,2700.99,2017-06-08T00:09:00,1.70000,4591.20
0,2703.49,2703.49,0a250158-516a-4678-814f-6808cddcfc32,2700.01,2701.00,2017-06-08T00:10:00,36.45000,98452.07
0,2704.24,2704.24,eff37c7d-6a03-4f5a-b6b9-f0face4540ce,2700.66,2703.49,2017-06-08T00:11:00,7.92000,21393.08
0,2705.16,2711.57,becac908-ab0c-4797-9cbe-608100e8e846,2704.24,2704.24,2017-06-08T00:12:00,10.83000,29315.27
0,2705.05,2711.57,e31c1285-9c3c-4772-bde6-cc6c069cefa6,2705.00,2705.00,2017-06-08T00:13:00,2.73000,7405.44
0,2709.19,2711.57,4ebbf38c-3593-447a-aac4-527dc5d66ad8,2705.05,2705.05,2017-06-08T00:14:00,6.86000,18566.94
0,2708.72,2709.19,e8b02d66-6b39-44bc-a72d-3c8c2952c45e,2705.49,2709.19,2017-06-08T00:15:00,34.08000,92234.97
0,2711.58,2711.58,f825d964-b1eb-4ef0-97de-88d9c24ae4a6,2708.72,2708.72,2017-06-08T00:16:00,7.84000,21253.52
0,2713.22,2713.22,f1c24cce-7556-45ca-af26-7cadcdc313f8,2710.53,2711.58,2017-06-08T00:17:00,4.55000,12331.85
0,2713.22,2713.92,bdd63215-b517-41d4-a076-6a5ff11b7e9d,2711.58,2713.22,2017-06-08T00:18:00,2.81000,7624.57


### ThinkDB delete statement

In [353]:
r.db("COIN_PRICES").table("BTCUSD").delete().run()

{'deleted': 5,
 'errors': 0,
 'inserted': 0,
 'replaced': 0,
 'skipped': 0,
 'unchanged': 0}

In [109]:
r.db("COIN_PRICES").table("BTCUSD").insert([{'close': 235.4,
  'high': 235.45,
  'low': 235.01,
  'open': 235.45,
  'time': 1496839500,
  'volumefrom': 74.81,
  'volumeto': 17606.41},
 {'close': 234.59,
  'high': 235.4,
  'low': 234.59,
  'open': 235.4,
  'time': 1496839680,
  'volumefrom': 38.21,
  'volumeto': 8977.23}]).run()



NameError: name 'Timestamp' is not defined

### MongoDB part

In [175]:
from pymongo import MongoClient

client = MongoClient()

db = client.primer

BTCUSD_PRICES = db.btcusd_prices

BTCUSD_PRICES.insert_many([{'close': 0.4,
  'high': 235.45,
  'low': 235.01,
  'open': 235.45,
  'time': 1496839500,
  'volumefrom': 74.81,
  'volumeto': 17606.41},
 {'close': 1.59,
  'high': 235.4,
  'low': 234.59,
  'open': 235.4,
  'time': 1496839680,
  'volumefrom': 38.21,
  'volumeto': 8977.23}])

In [193]:
df_from_db_m.time.dtype

dtype('O')

In [195]:
df_from_db_m = pd.DataFrame()
for i in BTCUSD_PRICES.find():
    #display(pd.io.json.json_normalize(i))
    df_from_db_m = df_from_db_m.append(pd.io.json.json_normalize(i))
    print(df_from_db_m.describe())
    #df_from_db_m['time'] = pd.to_datetime(df_from_db_m['time'].astype(int), unit="s")
    
df_from_db_m.time.describe()
    

       close    high     low    open          time  volumefrom  volumeto
count    1.0    1.00    1.00    1.00  1.000000e+00        1.00      1.00
mean   235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
std      NaN     NaN     NaN     NaN           NaN         NaN       NaN
min    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
25%    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
50%    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
75%    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
max    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
       close    high     low    open          time  volumefrom  volumeto
count    2.0    2.00    2.00    2.00  2.000000e+00        2.00      2.00
mean   235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
std      0.0    0.00    0.00    0.00  0.000000e+00        0.00      0.00
min    235.4  235.45  235.01  235.45  1.496840e+09 

count    1.000000e+01
mean     1.496840e+09
std      7.589466e+01
min      1.496840e+09
25%      1.496840e+09
50%      1.496840e+09
75%      1.496840e+09
max      1.496840e+09
Name: time, dtype: float64